# SageMaker Model Parallelism library を使用した日本語 GPT-NeoX の Instruction Tuning

<!-- In this notebook, you learn how to train or fine-tune the Hugging Face Transformers [GPT-NeoX-20B](https://huggingface.co/docs/transformers/model_doc/gpt_neox) model with the [Sharded Data Parallelism](https://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel-extended-features-pytorch-sharded-data-parallelism.html) technique in [SageMaker's Model Parallelism library (SMP)](https://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel.html) with PyTorch 1.13 and [GLUE/SST2 dataset](https://huggingface.co/datasets/glue/viewer/sst2/train) on SageMaker. 

Sharded data parallelism is a distributed training technique that splits the model parameters, gradients, and optimizer states across GPUs in a data parallel group. It is purpose-built for extreme-scale models and leverages Amazon in-house [MiCS](https://arxiv.org/pdf/2205.00119.pdf) technology which achieves a near-linear scaling efficiency. For large models that cannot fit into a single GPU, we recommend to use the sharded data parallelism technique with [Activation Checkpointing](https://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel-extended-features-pytorch-activation-checkpointing.html) and [Activation Offloading](https://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel-extended-features-pytorch-activation-offloading.html) in SMP first, before leveraging other techniques such as tensor parallelism or pipeline parallelism. -->

このノートブックでは、PyTorch 1.13 と SageMaker 上の [GLUE/SST2データセット](https://huggingface.co/datasets/glue/viewer/sst2/train) を使用して、[SageMakerのモデル並列化ライブラリ(SMP)](https://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel.html)の [Sharded Data Parallelism](https://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel-extended-features-pytorch-sharded-data-parallelism.html) 手法で [rinna/japanese-gpt-neox-3.6b](https://huggingface.co/rinna/japanese-gpt-neox-3.6b) モデルを学習または微調整する方法を学びます。

シャードデータ並列（Sharded Data Parallelism） は、モデルのパラメータ、勾配、オプティマイザの状態をデータ並列グループの GPU に分割する分散学習手法です。これは、極端なスケールのモデル用に構築されており、Amazon 社内の [MiCS](https://arxiv.org/pdf/2205.00119.pdf) 技術を活用して、ほぼ直線的なスケーリング効率を実現しています。1つの GPU に収まらない大規模なモデルの場合、テンソル並列やパイプライン並列などの他の技術を利用する前に、まず SMP で [Activation Checkpointing](https://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel-extended-features-pytorch-activation-checkpointing.html) と [Activation Offloading](https://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel-extended-features-pytorch-activation-offloading.html) を使用したシャーデッドデータ並列技術を使用することをお勧めします。

<!-- This notebook is accompanied with the following files:

- `train.py`: The entry point script that'll be passed to the SageMaker PyTorch estimator later in this notebook when launching the training job. This script is prepared to run an end-to-end training of the GPT-NeoX-20B model with SMP, settings for sharded data parallelism applied, and implemented with code lines to save, load, and fine-tune the model. You can follow the comments throughout the script to learn where the SMP APIs and code modifications are implemented.
- `data_pipeline.py`: This has data pipeline functions to prepare the training dataset.
- `learining_rate.py`: This has functions for learning rate schedule.
- `requirements.txt`: This installs the dependencies, including huggingface transformers.
- `memory_tracker.py`: This has functions to track memory usage.
- `model_config.py`: This has functions to get model configuration information.
- `sdp_utils.py`: This has util functions for sharded data parallelism -->

このノートブックには以下のファイルが付属しています：

- `train.py`： このノートブックの後半で学習ジョブを起動する際に SageMaker PyTorch estimator に渡されるエントリーポイントスクリプトです。このスクリプトは、日本語 GPT-NeoX モデルのエンドツーエンドのトレーニングを SMP で実行するために用意されており、シャードデータ並列化の設定が適用され、モデルの保存、ロード、微調整を行うコード行が実装されています。SMP API とコードの修正がどこに実装されているかは、スクリプト中のコメントをたどってください。
- `data_pipeline.py`： トレーニングデータセットを準備するためのデータパイプライン関数です。
- `learining_rate.py`： 学習レートのスケジュールに関する関数です。
- `requirements.txt`： huggingface transformers を含む依存関係をインストールします。
- `memory_tracker.py`： メモリ使用量を追跡する関数があります。
- `model_config.py`： モデルの設定情報を取得する関数です。
- `sdp_utils.py`： シャードデータ並列化のための utility 関数を提供します。

### 学習リソース

<!-- - To learn more about the SageMaker model parallelism library, see [Model Parallel Distributed Training with SageMaker Distributed](https://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel.html).

- To learn more about using the SageMaker Python SDK with PyTorch, see [Using PyTorch with the SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html).

- To learn more about launching a training job in Amazon SageMaker with your own training image, see [Use Your Own Training Algorithms](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html).

- To learn more about sharded data parallelism, check [Sharded Data Parallelism](https://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel-extended-features-pytorch-sharded-data-parallelism.html) or the blog [Near-linear scaling of gigantic-model training on AWS](https://www.amazon.science/blog/near-linear-scaling-of-gigantic-model-training-on-aws). -->

- 最適な構成、設定については、[大規模言語モデルを Amazon SageMaker 上で学習する際のベストプラクティス](https://aws.amazon.com/jp/blogs/news/training-large-language-models-on-amazon-sagemaker-best-practices/)を参照してください。

- SageMakerのモデル並列化ライブラリの詳細については、[SageMaker Distributedによるモデル並列分散トレーニング](https://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel.html)を参照してください。

- SageMaker Python SDK と PyTorch の併用については、[Using PyTorch with the SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/using_pytorch.html) を参照してください。

- Amazon SageMakerで独自のトレーニングイメージを使ってトレーニングジョブを起動する方法については、[Use Your Own Training Algorithms](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html)を参照してください。

- シャード化されたデータ並列化については、[Sharded Data Parallelism](https://docs.aws.amazon.com/sagemaker/latest/dg/model-parallel-extended-features-pytorch-sharded-data-parallelism.html) またはブログ [Near-linear scaling of gigantic-model training on AWS](https://www.amazon.science/blog/near-linear-scaling-of-gigantic-model-training-on-aws) をご覧ください。

### 事前準備

<!-- You must create an S3 bucket to store the input data for training. This bucket must be located in the same AWS Region that you choose to launch your training job. To learn how to create a S3 bucket, see [Create your first S3 bucket](https://docs.aws.amazon.com/AmazonS3/latest/userguide/creating-bucket.html) in the *Amazon S3 documentation*. -->

トレーニング用の入力データを保存するS3バケットを作成する必要があります。このバケットは、トレーニングジョブを起動するために選択したのと同じ AWS リージョンに配置する必要があります。S3 バケットの作成方法については、*Amazon S3 documentation*の[Create your first S3 bucket](https://docs.aws.amazon.com/AmazonS3/latest/userguide/creating-bucket.html)を参照してください。


## Amazon SageMaker 初期化

以下のセルを実行して、SageMaker モジュールをインポートし、AWS アカウント ID、AWS リージョン、Amazon SageMaker 実行ロールの ARN など、現在の SageMaker 作業環境の情報を取得します。SageMaker SDK を最新バージョンにアップグレードします。

カーネルの再起動が必要な場合があります。

In [ ]:
%pip install --upgrade sagemaker
%pip install sagemaker-experiments
%pip install sentencepiece

In [ ]:
%%time
import os

import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch

role = (
    get_execution_role()
)  # provide a pre-existing role ARN as an alternative to creating a new role
# role = "SageMakerRole"
print(f"SageMaker Execution Role: {role}")

client = boto3.client("sts")
account = client.get_caller_identity()["Account"]
print(f"AWS account: {account}")

session = boto3.session.Session()
region = session.region_name
print(f"AWS region: {region}")

sm_boto_client = boto3.client("sagemaker")
sagemaker_session = sagemaker.session.Session(boto_session=session)

# get default bucket
default_bucket = sagemaker_session.default_bucket()
print()
print("Default bucket for this session: ", default_bucket)

## データセットの準備

<!-- Here you will download, prepare the GLUE/SST2 dataset and then copy the files to S3. -->

### Install the Hugging Face Transformers and Datasets libraries

In [ ]:
!pip install -q datasets transformers==4.21.0

In [ ]:
import datasets
from datasets import load_dataset, load_from_disk, load_metric

In [ ]:
from sagemaker.pytorch import PyTorch
import transformers
import logging

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)

from transformers.testing_utils import CaptureLogger

In [ ]:
logger = logging.getLogger(__name__)

### Load data
<!-- This section loads the [GLUE/SST2](https://huggingface.co/datasets/glue/viewer/sst2/train) dataset and splits it to training and validation datasets. -->

[日本語の Dolly データセット](https://huggingface.co/datasets/kunishou/databricks-dolly-15k-ja)を例として使用します

In [ ]:
hyperparameters = {
    "dataset_name": "kunishou/databricks-dolly-15k-ja",
    "dataset_config_name": "train",
    "do_train": True,
    "do_eval": True,
    "cache_dir": "tmp",
}

In [ ]:
raw_datasets = load_dataset(
    hyperparameters["dataset_name"],
    hyperparameters["dataset_config_name"],
)

In [ ]:
if "validation" not in raw_datasets.keys():
    raw_datasets["validation"] = load_dataset(
        hyperparameters["dataset_name"],
        hyperparameters["dataset_config_name"],
        split="train[:5%]",
        cache_dir=hyperparameters["cache_dir"],
    )

    raw_datasets["train"] = load_dataset(
        hyperparameters["dataset_name"],
        hyperparameters["dataset_config_name"],
        split="train[5%:]",
        cache_dir=hyperparameters["cache_dir"],
    )

### Load tokenizer

<!-- Nearly every NLP task begins with a tokenizer. A tokenizer converts your text data into a format (token) that can be processed by the NLP model.
The following cell loads a tokenizer for GPT-NeoX-20B using [AutoTokenizer.from_pretrained()](https://huggingface.co/docs/transformers/v4.19.4/en/autoclass_tutorial#autotokenizer). -->

ほぼすべての NLP タスクはトークナイザーから始まります。トークナイザーは、テキスト・データを NLP モデルが処理できる形式（トークン）に変換します。
次のセルは、[AutoTokenizer.from_pretrained()](https://huggingface.co/docs/transformers/v4.19.4/en/autoclass_tutorial#autotokenizer) を使用して 日本語 GPT-NeoX 用のトークナイザをロードします。

In [ ]:
tokenizer_name = "rinna/japanese-gpt-neox-3.6b"
# tokenizer_name = "cyberagent/open-calm-1b"

tokenizer_kwargs = {
    "cache_dir": hyperparameters["cache_dir"],
}

tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, **tokenizer_kwargs)

### Preprocess data

データ形式を確認します

In [ ]:
print(raw_datasets["train"][0])
print(raw_datasets["train"][1])

Instruction 形式に変換する Prompter を定義します

In [ ]:
from typing import Union

class Prompter(object):

    def __init__(self, prompt_input: str = "", prompt_no_input: str = ""):
        self.template = {
            "prompt_input": prompt_input,
            "prompt_no_input": prompt_no_input
        }

    def generate_prompt(
        self,
        instruction: str,
        input: Union[None, str] = None,
        label: Union[None, str] = None,
    ) -> str:
        if input:
            res = self.template["prompt_input"].format(
                instruction=instruction, input=input
            )
        else:
            res = self.template["prompt_no_input"].format(
                instruction=instruction
            )
        if label:
            res = f"{res}{label}"
        return res
    
prompter = Prompter(
    prompt_input="Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n",
    prompt_no_input="Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\n{instruction}\n\n### Response:\n"
)

# Test
data_point = raw_datasets["train"][0]
print(prompter.generate_prompt(
    data_point["instruction"],
    data_point["input"],
    data_point["output"]
))

In [ ]:
def generate_and_tokenize_prompt(data_point):
    full_prompt = prompter.generate_prompt(
        data_point["instruction"],
        data_point["input"],
        data_point["output"],
    )
    tokenized_full_prompt = tokenize(full_prompt)
    return tokenized_full_prompt

def tokenize(prompt, add_eos_token=True):
    # there's probably a way to do this with the tokenizer settings
    # but again, gotta move fast
    result = tokenizer(
        prompt,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    return {
        "input_ids": result["input_ids"],
        "attention_mask": result["attention_mask"],
    }

In [ ]:
column_names = raw_datasets["train"].column_names
# text_column_name = "text" if "text" in column_names else column_names[0]

# since this will be pickled to avoid _LazyModule error in Hasher force logger loading before tokenize_function
tok_logger = transformers.utils.logging.get_logger("transformers.tokenization_utils_base")

tokenized_datasets = raw_datasets.map(
    generate_and_tokenize_prompt,
    # batched=True,
    num_proc=1,
    remove_columns=column_names,
    desc="Running tokenizer on dataset",
)

# 長すぎる文を削除
cutoff_len = 1024
print("Dataset Size Before Filter: ", tokenized_datasets)
tokenized_datasets = tokenized_datasets.filter(lambda x: len(x['input_ids']) < cutoff_len)
print("Dataset Size After Filter: ", tokenized_datasets)

lm_datasets = tokenized_datasets

In [ ]:
# 変換後のデータの確認
print(len(lm_datasets["train"][0]["input_ids"]))
print(lm_datasets["train"][0])

<!-- Set additional hyperparameters and S3 paths for mapping the train and validation datasets properly depending on the phase (training or validation) of the training job in each epoch. -->

各エポックにおける訓練ジョブのフェーズ（訓練または検証）に応じて、訓練データセットと検証データセットを適切にマッピングするための追加のハイパーパラメータと S3 パスを設定する。

In [ ]:
if hyperparameters["do_train"]:
    if "train" not in tokenized_datasets:
        raise ValueError("--do_train requires a train dataset")
    train_dataset = lm_datasets["train"]


if hyperparameters["do_eval"]:
    if "validation" not in tokenized_datasets:
        raise ValueError("--do_eval requires a validation dataset")
    eval_dataset = lm_datasets["validation"]

In [ ]:
training_dataset_location = None
validation_dataset_location = None


if hyperparameters["do_train"]:
    train_dataset.to_json("./training.json")
    training_dataset_location = "s3://{}/dataset/instruction/train/".format(default_bucket)

if hyperparameters["do_eval"]:
    eval_dataset.to_json("./validation.json")
    validation_dataset_location = "s3://{}/dataset/instruction/validation/".format(default_bucket)

In [ ]:
if training_dataset_location is not None:
    command = "aws s3 cp ./training.json {}".format(training_dataset_location)
    os.system(command)

if validation_dataset_location is not None:
    command = "aws s3 cp ./validation.json {}".format(validation_dataset_location)
    os.system(command)

In [ ]:
if hyperparameters["do_train"]:
    command = "rm ./training.json"
    os.system(command)

if hyperparameters["do_eval"]:
    command = "rm ./validation.json"
    os.system(command)

In [ ]:
%store training_dataset_location
%store validation_dataset_location

In [ ]:
%store

## Specify Amazon S3 Bucket Paths

<!-- Here you need to specify the paths for training data to be used by your job. The bucket used must be in the same region as where training will run. In the cells above you downloaded the GLUE/SST2 training and validation split datasets and uploaded the json files in an S3 bucket in your account. This example will train on those json files.

After you successfully run this example tensor parallel training job, you can modify the S3 bucket to where your own dataset is stored. -->

ここでは、ジョブで使用するトレーニングデータのパスを指定する必要があります。使用するバケットは、トレーニングが実行されるのと同じリージョンにある必要があります。上のセルでは、トレーニング用と検証用の分割データセットをダウンロードし、アカウント内のS3バケットに json ファイルをアップロードしました。この例では、これらの json ファイルを用いて学習を行います。

このサンプルデータの実行に成功した後、S3 バケットを自分のデータセットが保存されている場所に変更して試してみてください。

In [ ]:
%store -r training_dataset_location
%store -r validation_dataset_location

# if you're bringing your own data, uncomment the following lines and specify the locations there
# training_dataset_location = YOUR_S3_BUCKET/training
# validation_dataset_location = YOUR_S3_BUCKET/validation

In [ ]:
s3_train_bucket = training_dataset_location
s3_test_bucket = validation_dataset_location

<!-- The following S3 bucket will store the output artifacts of the training job. You can modify this as needed. -->
以下のS3バケットに、トレーニングジョブの出力成果物が保存されます。必要に応じて変更してください。

In [ ]:
s3_output_bucket = f"s3://sagemaker-{region}-{account}/smp-tensorparallel-outputdir/"

## Define Data Channels for SageMaker Training Using Amazon S3

<!-- In this step, define SageMaker training data channels to the S3 buckets.   -->

このステップでは、SageMaker トレーニングデータチャンネルを S3 バケットに定義します。 

In [ ]:
# Set use_fsx to False by default
# Set below var to True if you want to use fsx (see next cell)
use_fsx = False
if not use_fsx:
    if s3_train_bucket != None:
        train = sagemaker.inputs.TrainingInput(
            s3_train_bucket, distribution="FullyReplicated", s3_data_type="S3Prefix"
        )
        data_channels = {"train": train}
    else:
        data_channels = {"train": mock_data}
    if s3_test_bucket != None:
        test = sagemaker.inputs.TrainingInput(
            s3_test_bucket, distribution="FullyReplicated", s3_data_type="S3Prefix"
        )
        data_channels["test"] = test
    else:
        data_channels["test"] = mock_data
    print(data_channels)

<!-- ## (Optional) Set Up and Use Amazon FSx for Data Channels and Checkpoints

While the previous option of using Amazon S3 is easier to setup, using an FSx can be beneficial for performance when dealing with large input sizes and large model sizes. If you are using models above 13B, checkpointing should be done using FSx. 

Please see the instructions from [Distributed Training of Mask-RCNN in Amazon SageMaker Using FSx](https://github.com/aws/amazon-sagemaker-examples/blob/master/advanced_functionality/distributed_tensorflow_mask_rcnn/mask-rcnn-scriptmode-fsx.ipynb) to create an FSx Lustre file system and import the dataset from the S3 bucket to your FSx file system. Note that the FSx file system must be created in a private subnet with internet gateway to ensure that training job has access to the internet. For general guidance on setting an FSx Lustre file system as data input channel, see [Configure Data Input Channel to Use Amazon FSx for Lustre](https://docs.aws.amazon.com/sagemaker/latest/dg/model-access-training-data.html#model-access-training-data-fsx). -->

## (オプション) データチャネルとチェックポイントのためにAmazon FSxをセットアップして使用する

Amazon S3 を使用する上記の方法はセットアップが簡単ですが、大きな入力サイズと大きなモデルサイズを扱う場合 FSx を使用することはパフォーマンスにとって有益です。13B以上のモデルを使用する場合、チェックポイントはFSxを使用して行う必要があります。

FSx Lustre ファイルシステムを作成し、S3 バケットから FSx ファイルシステムにデータセットをインポートする方法は、[Distributed Training of Mask-RCNN in Amazon SageMaker Using FSx](https://github.com/aws/amazon-sagemaker-examples/blob/master/advanced_functionality/distributed_tensorflow_mask_rcnn/mask-rcnn-scriptmode-fsx.ipynb) を参照してください。FSx ファイルシステムは、トレーニングジョブがインターネットにアクセスできるように、インターネットゲートウェイがあるプライベートサブネット内に作成する必要があることに注意してください。FSx Lustre ファイルシステムをデータ入力チャネルとして設定する一般的なガイダンスについては、[Configure Data Input Channel to Use Amazon FSx for Lustre](https://docs.aws.amazon.com/sagemaker/latest/dg/model-access-training-data.html#model-access-training-data-fsx) を参照してください。

In [ ]:
# Instructions obtained from:
# https://github.com/aws/amazon-sagemaker-examples/blob/master/advanced_functionality/distributed_tensorflow_mask_rcnn/mask-rcnn-scriptmode-fsx.ipynb

if use_fsx:
    from sagemaker.inputs import FileSystemInput

    # Specify FSx Lustre file system id.
    file_system_id = "<your-file-system-id>"

    # Specify the SG and subnet used by the FSX, these are passed to SM Estimator so jobs use this as well
    fsx_security_group_id = "<your-security-group-id>"
    fsx_subnet = "<your-subnet>"

    # Specify directory path for input data on the file system.
    # You need to provide normalized and absolute path below.
    # Your mount name can be provided by you when creating fsx, or generated automatically.
    # You can find this mount_name on the FSX page in console.
    # Example of fsx generated mount_name: "3x5lhbmv"
    base_path = "<your-mount-name>"

    # Specify your file system type.
    file_system_type = "FSxLustre"

    train = FileSystemInput(
        file_system_id=file_system_id,
        file_system_type=file_system_type,
        directory_path=base_path,
        file_system_access_mode="rw",
    )

    data_channels = {"train": train, "test": train}

<!-- ## Set hyperparameters, metric definitions, and MPI options
The following `hyperparameters` dictionary passes arguments to the training script (`train.py`) and set the model parallel configuration when creating the training job.

You can also add custom `mpi` flags. By default, we have `--mca btl_vader_single_copy_mechanism none` to remove unnecessary logs.

Next, we add a base metric definitions to enable the metric upload in SageMaker. You can add any further metric definitions.

Note that we add the `sharded_data_parallel_degree` parameter to the `hyperparameter` dictionary. This will be parsed and used when we configure a SageMaker PyTorch estimator to activate sharded data parallelism.

Also note that we add the `fine_tune` parameter that activates the code lines for fine-tuning in the script `train.py`. If you want to fine-tune, set to `fine_tune=1`. -->

## ハイパーパラメータ、メトリック定義、MPIオプションの設定

以下の `hyperparameters` 辞書はトレーニングスクリプト (`train.py`) に引数として渡され、トレーニングジョブのモデルの並列設定などに利用されます。

また、カスタムの `mpi` フラグを追加することもできます。デフォルトでは、不要なログを削除するために `--mca btl_vader_single_copy_mechanism none` としています。

次に、SageMaker でメトリックのアップロードを有効にするための基本メトリック定義を追加します。さらにメトリック定義を追加することができます。

`hyperparameter` 辞書に `sharded_data_parallel_degree` パラメータを追加することに注意してください。これは SageMaker の PyTorch estimator のシャーデッドデータの並列性を有効にするときに使用されます。

また、スクリプト `train.py` に微調整用のコード行を有効にする `fine_tune` パラメータを追加していることにも注意してください。微調整を行いたい場合は `fine_tune=1` を指定してください。

### ハイパーパラメーターの選定

参考までに異なる設定での学習スループットを掲載します。

| model      | instance type | instance count | activation checkpoint | batch size | GPU Utilization | GPUMemoryUtilization | Throughput (samples/second) |
| ---------- | ------------- | -: | ----: | ---: | ----: | ----: | -------: |
| Rinna 3.6B |  p4d.24xlarge |  1 |  True |    8 |  774% |  564% |   35.5   |
| Rinna 3.6B |  p4d.24xlarge |  1 |  True |   32 |  781% |  760% | **40.2** |
| Rinna 3.6B |  p4d.24xlarge |  1 |  True |   64 |   OOM |   OOM |    OOM   |
| Rinna 3.6B |  p4d.24xlarge |  1 | False |    8 |  722% |  671% |   20.3   |
| Rinna 3.6B |  p4d.24xlarge |  1 | False |   16 |   OOM |   OOM |    OOM   |

| model      | instance type | instance count | activation checkpoint | batch size | GPU Utilization | GPUMemoryUtilization | Throughput (samples/second) |
| ---------- | ------------- | -: | ----: | ---: | ----: | ----: | -------: |
| Rinna 3.6B |  p4d.24xlarge |  1 |  True |   32 |  781% |  760% |   40.2   |
| Rinna 3.6B |  p4d.24xlarge |  2 |  True |   32 |  799% |  710% |   79.8   |
| Rinna 3.6B |  p4d.24xlarge |  4 |  True |   32 |  799% |  676% |  155.8   |
| Rinna 3.6B |  p4d.24xlarge |  4 |  True |   40 |  800% |  778% |  155.8   |

異なる設定でスループットを検証し最適なパラメータを選択することを推奨します。上記の表は1サンプル 1024 トークンでの検証結果であり、1サンプルあたりのトークン数が異なる Instruction Tuning ではスループットが変わる可能性があります。

また、算出されたスループットと学習データ量から学習時間とコストを試算することが可能です。

例として GPT Neox 3.6B を Instruction Tuning する際の学習時間とコストの試算を示します。 * コストは現時点での価格です。最新のコスト情報については、[Amazon SageMaker Pricing](https://aws.amazon.com/sagemaker/pricing/) をご参照ください。

In [ ]:
# コスト試算

def calculate_cost(
    samples,
    instance_type = 'p4d.24xlarge',
    instance_count = 4,
    aggregate_throughput = 153.5,
    sample_size = 1024
):
    print(f"== Estimation using {instance_count} x {instance_type}")
    
    # samples = token / sample_size
    print(f"Number of Samples: {samples}")

    samples_per_sec = aggregate_throughput
    samples_per_hour = samples_per_sec * 3600
    duration_hour = samples / samples_per_hour
    duration_day = duration_hour / 24
    print(f"Training Duration = {duration_hour} hours = {duration_day} days")

    single_instance_cost = {
        'p4d.24xlarge': 32.7726,
    }[instance_type]
    cost = single_instance_cost * instance_count * duration_hour
    print(f"Training Cost: ${cost}")

# Dolly Dataset JP Train Dataset: 15000 samples
samples = 15000
epoch = 2
total_samples = samples * epoch
calculate_cost(total_samples, instance_type='p4d.24xlarge', instance_count=1, aggregate_throughput=40.2)

In [ ]:
hyperparameters = {
    "max_steps": 900,
    "seed": 12345,
    "fp16": 0,
    "bf16": 1,
    "lr": 2.0e-4,
    "lr_decay_iters": 125000,
    "min_lr": 0.00001,
    "lr-decay-style": "linear",
    "warmup": 0.01,
    "num_kept_checkpoints": 5,
    "checkpoint_freq": 1000,
    "logging_freq": 1,
    "save_final_full_model": 1,
    "delayed_param": 1,
    "use_distributed_transformer": 1,
    "offload_activations": 0,
    "gradient_accumulation": 1,
    "validation_freq": 100,
    "train_batch_size": 32,
    "val_batch_size": 4,
    "flash_attention": 1,
    "zipped_data": 0,
    # "epochs": 100,
    "epochs": 1,
    "fine_tune": 1, # Fine tuning
    "model_name": "cyberagent/open-calm-1b", # OpenCALM 1B のファインチューニング
    "tokenizer_name": tokenizer_name,
    "model_type": "gpt_neox",
    # parameters for sharded data parallelism
    # "sharded_data_parallel_degree": 32,
}

if use_fsx:
    # make sure to update paths for training-dir and test-dir based on the paths of datasets in fsx
    # If you want to resume training, set checkpoint-dir to the same path as a previous job.
    SM_TRAIN_DIR = "/opt/ml/input/data/train"
    hyperparameters["checkpoint-dir"] = f"{SM_TRAIN_DIR}/checkpointdir-job2"
    hyperparameters["model-dir"] = f"{SM_TRAIN_DIR}/modeldir-job2"
    hyperparameters["training-dir"] = f"{SM_TRAIN_DIR}/datasets/pytorch_gpt/train_synthetic"
    hyperparameters["test-dir"] = f"{SM_TRAIN_DIR}/datasets/pytorch_gpt/val_synthetic"

# The checkpoint path (hyperparameters['checkpoint-dir'] or checkpoint_s3_uri) is not unique per job.
# You need to modify as needed for different runs.
# If same path is used for unrelated runs, this may increase time when downloading unnecessary checkpoints,
# and cause conflicts when loading checkpoints.

mpioptions = "-x NCCL_DEBUG=WARN -x SMDEBUG_LOG_LEVEL=ERROR "
mpioptions += (
    "-x SMP_DISABLE_D2D=1 -x SMP_D2D_GPU_BUFFER_SIZE_BYTES=1 -x SMP_NCCL_THROTTLE_LIMIT=1 "
)
mpioptions += "-x FI_EFA_USE_DEVICE_RDMA=1 -x FI_PROVIDER=efa -x RDMAV_FORK_SAFE=1"

metric_definitions = [
    # {"Name": "base_metric", "Regex": "<><><><><><>"}
    {'Name': 'eval_loss', 'Regex': "Validation loss: (\d\.\d+)"},
    {'Name': 'eval_perplexity', 'Regex': "Validation perplexity: (\d\.\d+)"},
    {'Name': 'train_loss', 'Regex': "Loss: (\d\.\d+)"}
]  # Add your custom metric definitions

<!-- Set the model configuration. -->
モデルの設定を行います。

必要に応じて以下のパラメータを変更してください。

In [ ]:
# TODO: Choose Model

model_config = "japanese-gpt-neox-3-6b"

if model_config == "gpt-neox-20b":
    model_params = {
        "max_context_width": 2048,
        "hidden_width": 6144,
        "num_layers": 44,
        "num_heads": 64,
    }
elif model_config == "japanese-gpt-neox-3-6b":
    model_params = {
        "max_context_width": 2048,
        "hidden_width": 2816,
        "num_layers": 36,
        "num_heads": 22,
        "rotary_pct": 1.0,
        "vocab_size": 32000
    }
elif model_config == "opencalm-1b":
    model_params = {
        "max_context_width": 2048,
        "hidden_width": 2048,
        "num_layers": 24,
        "num_heads": 16,
        "rotary_pct": 1.0,
        "vocab_size": 52096
    }
else:
    raise RuntimeError("Unknown model config")

for k, v in model_params.items():
    hyperparameters[k] = v

<!-- ## Specify Essential Parameters for a SageMaker Training Job

Next, you use the [`SageMaker Estimator class`](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html) to define a SageMaker Training Job, passing values through the following parameters for training job name, the number of EC2 instances, the instance type, and the size of the volume attached to the instances. 

* `instance_count`
* `instance_type`
* `volume_size`
* `base_job_name`

### Update the Type and Number of EC2 Instance to Use

The instance type and the number of instances you specify to the `instance_type` and `instance_count` parameters, respectively, determine the total number of GPUs (world size).

$$ \text{(world size) = (the number of GPUs on a single instance)}\times\text{(the number of instances)}$$ -->

## SageMaker トレーニングジョブに必要なパラメータの指定

次に、[`SageMaker Estimator class`](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html)を使用して、SageMaker トレーニングジョブを定義します。トレーニングジョブ名、EC2インスタンスの数、インスタンスタイプ、インスタンスにアタッチされるボリュームのサイズを以下のパラメータで指定します。

* `instance_count`
* `instance_type`
* `volume_size`
* `base_job_name`

### 使用するEC2インスタンスのタイプと数を更新する。

`instance_type` と `instance_count` パラメータにそれぞれ指定するインスタンスタイプとインスタンス数によって、GPUの総数（ワールドサイズ）が決まります。

$$ \text{(world size) = (the number of GPUs on a single instance)}\times\text{(the number of instances)}$$

In [ ]:
# TODO: Change Instance Type and count

instance_type = "ml.p4d.24xlarge"
# instance_type = "ml.p3.16xlarge"
instance_count = 1

# set to the number of GPUs on that instance
processes_per_host = {
    "ml.p3.8xlarge": 4,
    "ml.p3.16xlarge": 8,
    "ml.p4d.24xlarge": 8
}[instance_type]

gpu_num = instance_count * processes_per_host
hyperparameters["sharded_data_parallel_degree"] = gpu_num
print(hyperparameters["sharded_data_parallel_degree"])

<!-- To look up the number of GPUs of different instance types, see [Amazon EC2 Instance Types](https://aws.amazon.com/ec2/instance-types/). Use the section **Accelerated Computing** to see general purpose GPU instances. Note that, for example, a given instance type `p4d.24xlarge` has a corresponding instance type `ml.p4d.24xlarge` in SageMaker.
For SageMaker supported `ml` instances and cost information, see [Amazon SageMaker Pricing](https://aws.amazon.com/sagemaker/pricing/).  -->

異なるインスタンスタイプのGPU数を調べるには、[Amazon EC2 Instance Types](https://aws.amazon.com/ec2/instance-types/)を参照してください。汎用 GPU インスタンスを見るには、**Accelerated Computing** セクションをご参照ください。例えば、`p4d.24xlarge` インスタンスタイプは、SageMaker では `ml.p4d.24xlarge` というインスタンスタイプに対応しています。
SageMaker がサポートする `ml` インスタンスとコスト情報については、[Amazon SageMaker Pricing](https://aws.amazon.com/sagemaker/pricing/) を参照してください。

### Specify a Base Job Name

In [ ]:
machine_str = instance_type.split(".")[1] + instance_type.split(".")[2][:3]
sharding_degree = hyperparameters["sharded_data_parallel_degree"]
base_job_name = (
    f'smp-{model_config}-{machine_str}-sdp{sharding_degree}-bs{hyperparameters["train_batch_size"]}'
)

In [ ]:
if not use_fsx:
    # If you want to resume training, set checkpoint_s3_uri to the same path as a previous job.
    # Previous checkpoint to load must have same model config.
    checkpoint_bucket = f"s3://sagemaker-{region}-{account}"
    checkpoint_s3_uri = (
        f"{checkpoint_bucket}/experiments/gpt_synthetic_simpletrainer_checkpoints/{base_job_name}/"
    )

In [ ]:
print(f"base_job_name: {base_job_name} checkpoint_s3_uri: {checkpoint_s3_uri}")

### Create a SageMaker PyTorch Estimator

<!-- The following cell constructs a PyTorch estimator using the parameters defined above. To see how the SageMaker APIs and functions are applied to the script, see the `train.py` file. -->

次のセルは、上記で定義したパラメータを使用して PyTorch Estimator を構築します。SageMaker の API と関数がどのようにスクリプトに適用されているかは `train.py` ファイルを参照してください。

In [ ]:
kwargs = {}
if use_fsx:
    # Use the security group and subnet that was used to create the fsx filesystem
    kwargs["security_group_ids"] = [fsx_security_group_id]
    kwargs["subnets"] = [fsx_subnet]

smp_estimator = PyTorch(
    entry_point="train.py",
    source_dir="scripts",
    role=role,
    instance_type=instance_type,
    instance_count=instance_count,
    sagemaker_session=sagemaker_session,
    distribution={
        "mpi": {
            "enabled": True,
            "processes_per_host": processes_per_host,
            "custom_mpi_options": mpioptions,
        },
        "smdistributed": {
            "modelparallel": {
                "enabled": True,
                "parameters": {
                    "ddp": True,
                    "skip_tracing": True,
                    "delayed_parameter_initialization": hyperparameters["delayed_param"] > 0,
                    "offload_activations": hyperparameters["offload_activations"] > 0,
                    "sharded_data_parallel_degree": hyperparameters["sharded_data_parallel_degree"],
                    "fp16": hyperparameters["fp16"] > 0,
                    "bf16": hyperparameters["bf16"] > 0,
                    # partitions is a required param in the current SM SDK so it needs to be passed,
                    "partitions": 1,
                },
            }
        },
    },
    framework_version="1.13",
    py_version="py39",
    output_path=s3_output_bucket,
    checkpoint_s3_uri=checkpoint_s3_uri if not use_fsx else None,
    checkpoint_local_path=hyperparameters["checkpoint-dir"] if use_fsx else None,
    metric_definitions=metric_definitions,
    hyperparameters=hyperparameters,
    debugger_hook_config=False,
    disable_profiler=True,
    base_job_name=base_job_name + "-instruct",
    # keep_alive_period_in_seconds=600,
    **kwargs,
)

<!-- Finally, run the `estimator.fit` method to launch the SageMaker training job of the GPT-NeoX-20B model with sharded data parallelism. -->

最後に、`estimator.fit` メソッドを実行して、モデルの SageMaker トレーニングジョブを起動します。

In [ ]:
smp_estimator.fit(inputs=data_channels, logs=True)

## Download and inspect Artifact

In [ ]:
!aws s3 cp {smp_estimator.model_data} neox_3_6b_instruct.tar.gz
# !aws s3 cp {smp_estimator.model_data} neox_1b_instruct.tar.gz

In [ ]:
!mkdir -p neox_3_6b_instruct
!tar -xvf neox_3_6b_instruct.tar.gz -C neox_3_6b_instruct --no-same-owner
!ls -l neox_3_6b_instruct

# !mkdir -p neox_1b_instruct
# !tar -xvf neox_1b_instruct.tar.gz -C neox_1b_instruct --no-same-owner
# !ls -l neox_1b_instruct

<!-- ## Accessing the Training Logs

You can access the training logs from [Amazon CloudWatch](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/WhatIsCloudWatch.html). Make sure to look at the logs of **algo-1** because that is the main node whose output stream has the training job logs.

You can use CloudWatch to track SageMaker GPU and memory utilization during training and inference. To view the metrics and logs that SageMaker writes to CloudWatch, see [SageMaker Jobs and Endpoint Metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/monitoring-cloudwatch.html#cloudwatch-metrics-jobs) in the Amazon SageMaker Developer Guide.

If you are a new user of CloudWatch, see [Getting Started with Amazon CloudWatch](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/GettingStarted.html). 

For additional information on monitoring and analyzing Amazon SageMaker training jobs, see [Monitor and Analyze Training Jobs Using Metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/training-metrics.html).

## Deploying Trained Model for Inference

In most cases, a trained model can be deployed on a single device for inference because inference only requires a small amount of memory. You can use the SMP API to create a single, unified model after training: the [smp.DistributedModel.save_model()](https://sagemaker.readthedocs.io/en/stable/api/training/smp_versions/latest/smd_model_parallel_tensorflow.html#smp.DistributedModel.save_model) method for TensorFlow, and the [smp.save()](https://sagemaker.readthedocs.io/en/stable/api/training/smp_versions/latest/smd_model_parallel_pytorch.html#apis-for-saving-and-loading) function for PyTorch.

After you build and train your models, you can deploy them to get predictions in one of two ways:

* To set up a persistent endpoint to get predictions from your models, use SageMaker hosting services. For an overview on deploying a single model or multiple models with SageMaker hosting services, see [Deploy a Model on SageMaker Hosting Services](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-deployment.html#how-it-works-hosting).
* To get predictions for an entire dataset, use SageMaker batch transform. For an overview on deploying a model with SageMaker Batch Transform, see [Get Inferences for an Entire Dataset with Batch Transform](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html).

To learn more about deploying models for inference using SageMaker, see [Deploy Models for Inference](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html). 
 -->
 
## トレーニングログへのアクセス

[Amazon CloudWatch](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/WhatIsCloudWatch.html) からトレーニングログにアクセスできます。

CloudWatch を使用すると、学習と推論中の SageMaker GPU とメモリの使用状況を追跡できます。SageMaker が CloudWatch に書き込むメトリクスとログを見るには、Amazon SageMaker Developer Guide の[SageMaker Jobs and Endpoint Metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/monitoring-cloudwatch.html#cloudwatch-metrics-jobs) を参照してください。

CloudWatch を初めて使用する場合は、[Getting Started with Amazon CloudWatch](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/GettingStarted.html) を参照してください。

Amazon SageMaker トレーニングジョブの監視と分析に関する追加情報については、[Monitor and Analyze Training Jobs Using Metrics](https://docs.aws.amazon.com/sagemaker/latest/dg/training-metrics.html) を参照してください。

## 推論のための学習済みモデルのデプロイ

ほとんどの場合、学習済みモデルは推論のために1つのデバイス上に配置することができます。TensorFlow では[smp.DistributedModel.save_model()](https://sagemaker.readthedocs.io/en/stable/api/training/smp_versions/latest/smd_model_parallel_tensorflow.html#smp.DistributedModel.save_model)メソッド、PyTorchでは[smp.save()](https://sagemaker.readthedocs.io/en/stable/api/training/smp_versions/latest/smd_model_parallel_pytorch.html#apis-for-saving-and-loading)関数を使用します。

モデルをビルドしてトレーニングした後、2つの方法のいずれかで推論のためにデプロイすることができます：

* モデルから予測値を取得するための永続的なエンドポイントを設定するには、SageMaker ホスティングサービスを使用します。SageMaker ホスティングサービスを使用した単一モデルまたは複数モデルのデプロイの概要については、[SageMaker ホスティングサービスでのモデルのデプロイ](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-deployment.html#how-it-works-hosting) を参照してください。
* データセット全体の予測値を取得するには、SageMaker バッチ変換を使用します。SageMaker バッチ変換を使用したモデルのデプロイの概要については、[バッチ変換でデータセット全体の推論を取得する](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html) を参照してください。

SageMaker を使った推論のためのモデルのデプロイについては、[推論のためのモデルのデプロイ](https://docs.aws.amazon.com/sagemaker/latest/dg/deploy-model.html) を参照してください。
